In [1]:
from rdkit.Chem.Descriptors import *
from rdkit import Chem
import pandas as pd
from rdkit.Chem import Descriptors
import sys
from rdkit import Chem
from rdkit.Chem import AllChem, TorsionFingerprints
from rdkit.ML.Cluster import Butina

In [2]:
data = pd.read_csv("sampleMolecules.csv")

In [3]:
SmilesCode = data['canonicalSmiles'].values

In [4]:
SmilesCode[5]

'CNC(=O)C1(C)CCC2(C)CCC3(C)C4=CC=C5C(=CC(=O)C(O)=C5C)C4(C)CCC3(C)C2C1'

In [5]:
def formSMILESintoMOL(SmilesCode):
    m = Chem.MolFromSmiles(SmilesCode)
    return Descriptors.MolWt(m),Descriptors.Chem.AddHs(m),MolWt(m)

In [6]:
SmilesCode[2]

'CC(C)c1cccc(C(C)C)c1-n1sc2ncccc2c1=O'

In [7]:
formSMILESintoMOL(SmilesCode[2])

(312.43800000000005, <rdkit.Chem.rdchem.Mol at 0x104eea70>, 312.43800000000005)

In [8]:
def formSMILESintoMOL(SmilesCode):
    m = Chem.MolFromSmiles(SmilesCode)
    return Descriptors.MolWt(m)
    m_H = Descriptors.Chem.AddHs(SmilesCode)
    return Descriptors.MolWt(m_H)

In [9]:
formSMILESintoMOL(SmilesCode[3])

568.4330000000001

In [10]:
formSMILESintoMOL(SmilesCode[2])

312.43800000000005

In [11]:
def formSMILESintoMOL(SmilesCode):
    m = Chem.MolFromSmiles(SmilesCode)
    return Descriptors.MolWt(m)
def addHs(SmilesCode):
    m_H = Descriptors.Chem.AddHs(SmilesCode)
    return Descriptors.MolWt(m_H)

In [12]:
formSMILESintoMOL(SmilesCode[3]),addHs(SmilesCode[3])

ArgumentError: Python argument types in
    rdkit.Chem.rdmolops.AddHs(str)
did not match C++ signature:
    AddHs(class RDKit::ROMol mol, bool explicitOnly=False, bool addCoords=False, class boost::python::api::object onlyOnAtoms=None)

In [21]:
def formSMILESintoMOL(SmilesCode):
    m = Chem.MolFromSmiles(SmilesCode)
    return Descriptors.MolWt(m)

In [25]:
formSMILESintoMOL(SmilesCode[2])

312.43800000000005

In [30]:
results = []
for item in SmilesCode:
    descriptors = formSMILESintoMOL(item)
    results.append([item, descriptors[1]])

TypeError: 'float' object has no attribute '__getitem__'

In [19]:
def gen_conformers(SmilesCode, numConfs=100, maxAttempts=1000, pruneRmsThresh=0.1, useExpTorsionAnglePrefs=True, useBasicKnowledge=True, enforceChirality=True):
    ids = AllChem.EmbedMultipleConfs(mol, numConfs=numConfs, maxAttempts=maxAttempts, pruneRmsThresh=pruneRmsThresh, useExpTorsionAnglePrefs=useExpTorsionAnglePrefs, useBasicKnowledge=useBasicKnowledge, enforceChirality=enforceChirality, numThreads=0)
    return list(ids)

In [20]:
gen_conformers(SmilesCode[1])

ArgumentError: Python argument types in
    rdkit.Chem.rdDistGeom.EmbedMultipleConfs(str)
did not match C++ signature:
    EmbedMultipleConfs(class RDKit::ROMol {lvalue} mol, unsigned int numConfs, struct RDKit::DGeomHelpers::EmbedParameters {lvalue} params)
    EmbedMultipleConfs(class RDKit::ROMol {lvalue} mol, unsigned int numConfs=10, unsigned int maxAttempts=0, int randomSeed=-1, bool clearConfs=True, bool useRandomCoords=False, double boxSizeMult=2.0, bool randNegEig=True, unsigned int numZeroFail=1, double pruneRmsThresh=-1.0, class boost::python::dict {lvalue} coordMap={}, double forceTol=0.001, bool ignoreSmoothingFailures=False, bool enforceChirality=True, int numThreads=1, bool useExpTorsionAnglePrefs=False, bool useBasicKnowledge=False, bool printExpTorsionAngles=False)

NameError: name 'ids' is not defined

In [16]:
def write_conformers_to_sdf(mol, filename, rmsClusters, conformerPropsDict, minEnergy):
	w = Chem.SDWriter(filename)
	for cluster in rmsClusters:
		for confId in cluster:
			for name in mol.GetPropNames():
				mol.ClearProp(name)
			conformerProps = conformerPropsDict[confId]
			mol.SetIntProp("conformer_id", confId + 1)
			for key in conformerProps.keys():
				mol.SetProp(key, str(conformerProps[key]))
			e = conformerProps["energy_abs"]
			if e:
				mol.SetDoubleProp("energy_delta", e - minEnergy)
			w.write(mol, confId=confId)
	w.flush()
	w.close()

In [17]:
def calc_energy(mol, conformerId, minimizeIts):
	ff = AllChem.MMFFGetMoleculeForceField(mol, AllChem.MMFFGetMoleculeProperties(mol), confId=conformerId)
	ff.Initialize()
	ff.CalcEnergy()
	results = {}
	if minimizeIts > 0:
		results["converged"] = ff.Minimize(maxIts=minimizeIts)
	results["energy_abs"] = ff.CalcEnergy()
	return results